In [10]:
import pandas as pd
import numpy as np
import random
import time
import argparse
import json
import array
from copy import deepcopy # Add Deepcopy for args

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error,r2_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


seed = 777
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
class RNN(nn.Module):
    ''' Sigle RNN model '''
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
    
        '''
        : param input_dim   :   the number of features in the input X
        : param hidden_dim  :   the number of features in the hidden state h
        : param output_dim  :   the number of valeus in y
        : param num_layers:     number of recurrent layers (i.e., 2 means there are 2 stacked LSTMs)
        '''
        
        super(RNN, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.dropout = dropout

        self.rnn = nn.GRU(self.input_dim, self.hidden_dim, self.num_layers)
        self.fc_out = self.make_regressor()
        
    def init_hidden(self, batch_size):
        ''' initialize hidden state '''
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))
    
    def forward(self, X):
        rnn_out, self.hidden = self.rnn(X)
        output = self.fc_out(rnn_out[-1].view(X.shape[1], -1)) 
        return output

    def make_regressor(self):
        layers = []
        layers.append(nn.Dropout(self.dropout))
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor

### TRAIN, VALIDATION, TEST

In [3]:
def train(model, partition, optimizer, loss_fn, args):
    ''' model training '''
    
    # data load
    trainloader = DataLoader(partition['train'], batch_size = args.batch_size, shuffle = True, drop_last = True)
    
    # model's mode setting
    model.train()

    # model initialization
    model.zero_grad()
    train_loss = 0.0
    
    # train
    for i, (X, y) in enumerate(trainloader):
        
        X = X.transpose(0, 1).float().to(args.device)
        y_true = y[:,:,:args.output_dim].transpose(0, 1).float().to(args.device)
        
        # zero the gradient
        optimizer.zero_grad()
        
        model.hidden = model.init_hidden(X.shape[1])

        # en-decoder outputs tensor 
        y_pred = model(X)
        
        # compute the loss 
        loss = loss_fn(y_true, y_pred)
        
        #backpropagation
        loss.backward()
        optimizer.step()
        
        # get the batch loss
        train_loss += loss.item()
        
    train_loss = train_loss / len(trainloader)
    return model, train_loss

In [4]:
def validate(model, partition, loss_fn, args):
    ''' model validate '''
    
    # data load
    valloader = DataLoader(partition['val'], batch_size = args.batch_size, shuffle = False, drop_last = True)
    
    # model's mode setting
    model.eval()
    val_loss = 0.0
    
    # evaluate
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:,:,:args.output_dim].transpose(0, 1).float().to(args.device)
            
            model.hidden = model.init_hidden(X.shape[1])
 
            # en-decoder outputs tensor 
            y_pred = model(X)

            # compute the loss 
            loss = loss_fn(y_true, y_pred)

            # get the batch loss
            val_loss += loss.item()
            
    val_loss = val_loss / len(valloader)
    return val_loss

In [15]:
def test(model, partition, scaler, args):
    ''' model test '''
    
    # data load
    testloader = DataLoader(partition['test'], batch_size = 1, shuffle = False, drop_last = False)
    
    # predict
    model.eval()

    # evaluate
    test_mae = 0.0
    score_list = list()
    item_loss_list = list()

    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            
            X = X.transpose(0, 1).float().to(args.device)            
            y_true = y[:,:,:args.output_dim].transpose(0, 1).float().to(args.device)
    
            y_pred = model(X)

            y_true = y_true.view(-1, args.output_dim)
            y_pred = y_pred.view(-1, args.output_dim)

            # y values to cpu
            y_true = y_true.cpu().detach().numpy()
            y_pred = y_pred.cpu().detach().numpy()

            # inverse trasform y valuse 
            y_true = scaler.inverse_transform(y_true).round()
            y_pred = scaler.inverse_transform(y_pred).round()
            
            print(y_true.shape, y_pred.shape)
            # get the batch loss
            test_mae += mean_absolute_error(y_true, y_pred)
            score = r2_score(y_pred = y_pred.transpose(), y_true = y_true.transpose(), multioutput = 'uniform_average')
            score_list.append(score)
            item_loss_list.append(abs(y_true - y_pred))


        test_mae /= len(testloader)
        score /= len(testloader)

    return test_mae, score_list, item_loss_list

In [6]:
def experiment(partition, scaler, args):

    # model
    model = RNN(args.input_dim, args.hidden_dim, args.output_dim, args.num_layers, args.dropout)
    model.to(args.device)
    
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.L2_rate)
    
    # epoch-wise loss
    train_losses = []
    val_losses = []
  
    for epoch in range(args.num_epoch):
        
        start_time = time.time()
        
        model, train_loss = train(model, partition, optimizer, loss_fn, args)
        val_loss = validate(model, partition, loss_fn, args)
        
        end_time = time.time()
        
        # add epoch loss
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print('Epoch {},Loss(train/val) {:.3f}/{:.3f}. Took {:.2f} sec'.format(epoch+1, train_loss, val_loss, end_time-start_time))
        
    # test part
    test_mae, score_list, item_loss_list = test(model, partition, scaler, args)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    
    result['train_losses'] = train_losses #epoch 수에 의존
    result['val_losses'] = val_losses 
    
    result['test_mae'] = test_mae.round(3).item()
    result['r2'] = np.array(score_list).mean().round(3)
    
    item_loss = np.array(item_loss_list).mean(axis=0).mean(axis=0).astype(int)
    item_loss = list([int(x) for x in item_loss])
    
    result['item_loss'] = item_loss
    
    return vars(args), result

In [12]:
# self-program
import data_handler
import generate_dataset
import plotting
import single_rnn

# data load
file_path = './total_mak.h5'
data = pd.read_hdf(file_path)

# 주말구분, 데이터 트렌드 추가
data = data_handler.trend_insert(data)
data = data_handler.weekend_insert(data)
data = data.dropna()

In [17]:
# ====== initialization
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is",args.device)

# ====== Model Capacity options ===== #
args.input_dim = len(data.columns)
args.hidden_dim = 50
args.output_dim = 7 # 상품군 아이템 개수
args.num_layers = 3 
args.batch_size = 64
args.dropout = 0.2
args.use_bn = True

# ====== Dataset Generating options ====== #
args.x_frames = 7
args.y_frames = 1
args.split_size = 0.8
args.test_days = 60

# ====== Model training options ===== #
args.num_epoch = 30
args.learning_rate = 0.0001
args.L2_rate = 0.0002

# ====== Data Loading ====== #
train_df, val_df, test_df = generate_dataset.train_val_test_split(dataframe = data.copy(), split_size = args.split_size, test_days = args.test_days)

# ====== Data scaling ====== # 
if len(data.columns) == args.output_dim:
    print("scaler v1\n")
    scaled_train_df, scaled_val_df, scaled_test_df, scaler1 = generate_dataset.standardnorm_scaler_v1(train_df, val_df, test_df)
    
else:
    print("scaler v2\n")
    scaled_train_df, scaled_val_df, scaled_test_df, scaler1, scaler2 = generate_dataset.standardnorm_scaler_v2(train_df, val_df, test_df, args.output_dim)
    

# ====== Data partition ====== #
trainset = generate_dataset.DatasetGenerater(scaled_train_df, x_frames = args.x_frames, y_frames = args.y_frames)
valset = generate_dataset.DatasetGenerater(scaled_val_df, x_frames = args.x_frames, y_frames = args.y_frames)
testset = generate_dataset.DatasetGenerater(scaled_test_df, x_frames = args.x_frames, y_frames = args.y_frames)
partition = {'train': trainset, 'val':valset, 'test':testset}

device is cuda
1092 274
Data is split normally.

train_len is 1092 days, validation_len is 274 days, test_len is 60 days.

scaler v2

null clear
null clear
null clear
